In [17]:
workspace()

In [10]:
using EvolvingGraphs
using EvolvingGraphs.Centrality
using StatsBase

# Katz Motivation exmple 

In [2]:
g = EvolvingGraph{Node{String}, Int}()

Directed EvolvingGraph 0 nodes, 0 static edges, 0 timestamps

In [3]:
add_bunch_of_edges!(g, [("A", "B", 1), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 3)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [4]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [6]:
katz(g)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.742301)
 (Node(B), 0.42943) 
 (Node(C), 0.514373)

In [90]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [5]:
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [91]:
g3 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g3, [("A", "B", 100), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])
katz(g3)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [7]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A","B", 1),("B","C", 2)])
katz(g1)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.64654) 
 (Node(B), 0.604677)
 (Node(C), 0.465136)

In [8]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

# Katz Centrality

Each node `v` has forword neighbours. Each forward neighbour has forword neighbour, and so on.

In [40]:
"""
distance between two active nodes

alpha: time unit parameter
"""
function temporal_distance(v1::TimeNode, v2::TimeNode, beta::Real = 1.)
    return beta* abs(node_timestamp(v1) - node_timestamp(v2))
end
function temporal_distance(v1::Tuple{Int,Int}, v2::Tuple{Int,Int}, beta::Real = 1.)
    return beta * abs(v1[2] - v2[2])
end



temporal_distance (generic function with 4 methods)

In [37]:
function temporal_katz(g::AbstractEvolvingGraph, start::TimeNode; alpha = 0.1, k = 10)
    score = 0.
    v = start
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                push!(next, v)
                td = temporal_distance(start, u)
                d = td + level
                score += alpha^d
            end
        end
        fronter = next
        level += 1
    end
    return score
end

function temporal_katz(g::IntAdjacencyList, start::Tuple{Int,Int}; alpha = 0.1, k = 10)
    score = 0.
    v = start
    fronter = [v]
    level = 0
    scale = num_edges(g)
    while level < k
        next = []
        for u in fronter
            fn = forward_neighbors(g, u)
            fn = sample(fn, min(100, length(fn)), replace=false)
            println("number of neighbors $(length(fn))")
            for v in fn
                push!(next, v)
                if v[1] != u[1]
                    td = temporal_distance(start, u)
                    d = td + level
                    score += alpha^d
                end
            end
        end
        fronter = next
        level += 1
    end
    return score/scale
end

# Can be done in parallel, focus on useful node at certain time stamp


temporal_katz (generic function with 2 methods)

In [13]:
active_nodes(g)

LoadError: [91mUndefVarError: g not defined[39m

In [8]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g, n, alpha = 0.1, k = 5)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(A, 1) => 2.22435
  TimeNode(A, 2) => 2.32421
  TimeNode(B, 1) => 2.0111
  TimeNode(B, 2) => 1.01
  TimeNode(B, 3) => 1.0
  TimeNode(C, 2) => 2.2324
  TimeNode(C, 3) => 0.0

In [27]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[node_key(n)] += v
end
sum_scores 

DataStructures.DefaultDict{Any,Any,Float64} with 3 entries:
  "B" => 4.25367
  "A" => 4.54167
  "C" => 2.77667

In [24]:
an = active_nodes(g2)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g2, n, alpha = 0.2, k = 10)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 2) => 1.0
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 3.58333
  TimeNode(C, 1) => 1.06867
  TimeNode(A, 3) => 1.0
  TimeNode(B, 1) => 3.25373
  TimeNode(C, 2) => 1.71667

In [26]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g3, n, alpha = 0.2, k = 10)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(C, 1)   => 1.06833
  TimeNode(B, 100) => 0.0
  TimeNode(C, 2)   => 1.70833
  TimeNode(B, 2)   => 1.0
  TimeNode(B, 1)   => 3.25367
  TimeNode(A, 100) => 1.0
  TimeNode(A, 2)   => 3.54167

# Communicability Betweenness Centrality 

In [1]:
"""
Take account of all forward neighours.
"""

function temporal_katz(g::AbstractEvolvingGraph, start_node::TimeNode, end_node::TimeNode; alpha = 0.2, k = 10)
    score = 0.
    v = start_node
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            fn = forward
            for v in forward_neighbors(g, u)
                push!(next, v)
                if u == end_node
                    td = temporal_distance(start_node, u)
                    d = td + level
                    score += alpha^d
                end
            end
        end
        fronter = next
        level += 1
    end
    return score
end


function temporal_resolvent_betweenness(g1::EvolvingGraph, g2::EvolvingGraph, v::TimeNode; alpha = 0.2, k = 10)
    r = 0.
    ns = active_nodes(g1)
    for i in ns
        for j in ns
            if i != j && i != v && j != v

                score1 = temporal_katz(g1, i, j, alpha = alpha, k = k)
                score2 = temporal_katz(g2, i, j, alpha = alpha, k = k)
#                 println("From node $i to node $j")
#                 println("score1 $score1 and score2 $score2")
                r += score1 == 0.0 ? 0 : (score1 - score2)/score1
            end
        end
    end
    return r
end

LoadError: [91mUndefVarError: AbstractEvolvingGraph not defined[39m

In [84]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A", "B", 100), ("A", "C", 10), ("A", "B", 10),("C", "A", 10),("B", "C", 1)])

g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 100), ("A", "B", 10),("B", "C", 1)])

v = active_nodes(g1)[4]

TimeNode(C, 10)

In [85]:
temporal_resolvent_betweenness(g1, g2, v; alpha = 0.2, k = 10)

6.088263741647286

In [86]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A", "B", 100), ("A", "C", 10), ("A", "B", 10),("C", "A", 10),("B", "C", 1)])

g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "C", 10), ("A", "B", 10),("C", "A", 10),("B", "C", 1)])

v = active_nodes(g1)[1]

TimeNode(A, 100)

In [88]:
temporal_resolvent_betweenness(g1, g2, v; alpha = 0.2, k = 10)

5.0

In [89]:
v = active_nodes(g1)[2]
println(v)
temporal_resolvent_betweenness(g1, g2, v; alpha = 0.2, k = 10)

TimeNode(B, 100)


9.0

In [14]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_communicability(g, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 0.0564789
  TimeNode(C, 3) => 0
  TimeNode(A, 1) => 0.0
  TimeNode(B, 2) => 0.0
  TimeNode(B, 1) => 0.0
  TimeNode(C, 2) => 0.0860369

In [12]:
0/0

NaN

# Temporal Betweenness

In constrast with Katz and Communicability (which are based on walks), the following centrality algorithms are based on shortest paths.

In [ ]:
#TODO later

# Temporal Closeness Centrality 

Closeness centrality (or closeness) of a node is a measure of centrality in a network, calculated as the sum of the length of the shortest paths between the node and all other nodes in the graph.

In [92]:
function temporal_closeness(g::Union{AbstractEvolvingGraph, IntAdjacencyList}, start::Union{TimeNode,Tuple{Int,Int}})
    v = start
    level = Dict(v => 0)
    i = 1
    fronter = [v]
    while length(fronter) > 0
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                if !(v in keys(level))
                    td = temporal_distance(start, u)
                    level[v] = i + td
                    push!(next, v)
                end
            end
        end
        fronter = next
        i += 1
    end
    
    total_scores = sum(values(level))
    return total_scores > 0. ? (length(level) - 1)/total_scores : 0.
end

temporal_closeness (generic function with 1 method)

In [16]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [93]:
an = active_nodes(g2)

scores = Dict()
for n in an
    scores[n] = temporal_closeness(g2, n)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 3) => 0.0
  TimeNode(B, 1) => 0.27027
  TimeNode(A, 3) => 1.0
  TimeNode(B, 2) => 1.0
  TimeNode(C, 1) => 0.259259
  TimeNode(A, 2) => 0.538462
  TimeNode(C, 2) => 0.416667

In [94]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = temporal_closeness(g3, n)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 100) => 0.0
  TimeNode(A, 2)   => 0.0636364
  TimeNode(A, 100) => 1.0
  TimeNode(C, 2)   => 0.0458716
  TimeNode(B, 1)   => 0.0746269
  TimeNode(C, 1)   => 0.0564516
  TimeNode(B, 2)   => 1.0

# Temporal PageRank

Random walk interpretation can not travel back in time.

* Block matrix version (use reverse pagerank): useful for evolving graph with a small number of timestamps

* Probability based (time-stamps based rating): Random walk interpretation. Not fixed probability for all nodes but different probability for each individual node. 

In [100]:
function block_google_matrix(g::IntAdjacencyList; alpha = 0.85, balance = 0.75)
    A = full(block_adjacency_matrix(g))
    A = balance * A + (1-balance) * A'
    N, N = size(A)
    p = ones(N)/N
    dangling_weights = p
    dangling_nodes = find(x->x==0, sum(A,2))
    
    for node in dangling_nodes
        A[node,:] = dangling_weights
    end
    A ./= sum(A,2)
    return alpha * A .+ (1- alpha) * p
end

block_google_matrix (generic function with 1 method)

In [101]:
g = IntAdjacencyList(4,3)
add_edge!(g, 1, 2, 1)
add_edge!(g, 2, 3, 1)
add_edge!(g, 1, 4, 2)

Directed IntAdjacencyList (4 nodes, 3 static edges, 3 timestamps)

In [102]:
A = google_matrix_block(g)
sum(A, 2)

LoadError: [91mUndefVarError: google_matrix_block not defined[39m

In [111]:
function temporal_pagerank(g::AbstractEvolvingGraph)
    A = block_google_matrix(g)
    F = eigfact(A')
    println("F values $(F[:values])")
    return F[:vectors][:,1]
end

temporal_pagerank (generic function with 1 method)

In [112]:
g = evolving_graph_to_adj(g2)
temporal_pagerank(g)

F values Complex{Float64}[1.0+0.0im, -0.676908+0.0im, -0.615626+0.0im, -0.275883+0.0im, 0.359208+0.0861858im, 0.359208-0.0861858im, 0.188889+0.0im, -4.94405e-17+0.0im, 2.62847e-17+0.0im]


9-element Array{Complex{Float64},1}:
   0.12278+0.0im
  0.254264+0.0im
 0.0533627+0.0im
   0.36275+0.0im
  0.350713+0.0im
  0.406792+0.0im
 0.0533627+0.0im
  0.524112+0.0im
  0.468848+0.0im

# Random Evolving Graph Data

Start with the original definition and compare the change of ranking of top authors if we vary

In [5]:
g = random_evolving_graph(500, 10, 0.2)

Directed IntAdjacencyList (500 nodes, 498932 static edges, 10 timestamps)

In [6]:
r = katz(g)
sorted_top = sort(r, by = x -> x[2], rev = true)[1:10]

10-element Array{Tuple{Int64,Float64},1}:
 (334, 0.152288) 
 (9, 0.137403)   
 (428, 0.126793) 
 (164, 0.121582) 
 (250, 0.117862) 
 (35, 0.117627)  
 (291, 0.105793) 
 (414, 0.0975953)
 (123, 0.096394) 
 (478, 0.0945754)

In [8]:
sorted_top_keys = map(x -> x[1], sorted_top);

In [20]:
intg = evolving_graph_to_adj(g)

Directed IntAdjacencyList (100 nodes, 9771 static edges, 5 timestamps)

In [41]:
scores = []

for n in sorted_top_keys
    for t in 1:10
        intn = (n, 5)
        tr = temporal_katz(g, intn, alpha = 0.2, k = 3)
        push!(scores, (intn, tr))
    end
end
scores

number of neighbors 100
number of neighbors 100
number of neighbors 88
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 91
number of neighbors 96
number of neighbors 100
number of neighbors 88
number of neighbors 100
number of neighbors 100
number of neighbors 99
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 97
number of neighbors 100
number of neighbors 95
number of neighbors 100
number of neighbors 98
number of neighbors 99
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
nu

number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 91
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 88
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 97
number of neighbors 93
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 83
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 89
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 97
n

number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 92
number of neighbors 94
number of neighbors 85
number of neighbors 100
number of neighbors 99
number of neighbors 95
number of neighbors 100
number of neighbors 92
number of neighbors 100
number of neighbors 100
number of neighbors 82
number of neighbors 97
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 95
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 95
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 91
number of neighbors 100
number of neighbors 100
numbe

number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 99
number of neighbors 88
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 90
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100

number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 96
number of neighbors 90
number of neighbors 100
number of neighbors 100
number of neighbors 94
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 95
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 100
number of neighbors 100
number of neighbors 99
number of neighbors 100
number of neighbors 100
n

...

number of neighbors 93
number of neighbors 100
number of neighbors 99
number of neighbors 93
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 100
number of neighbors 81
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 90
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 96
number of neighbors 100
number of neighbors 88
number of neighbors 100
number of neighbors 84
number of neighbors 100
number of neighbors 96
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
num

number of neighbors 98
number of neighbors 98
number of neighbors 100
number of neighbors 92
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 99
number of neighbors 100
number of neighbors 100
number of neighbors 91
number of neighbors 95
number of neighbors 100
number of neighbors 100
number of neighbors 95
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 88
number of neighbors 90
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 98
number of neighbors 100
number of neighbors 100
numbe

Excessive output truncated after 524305 bytes.

number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 94
number of neighbors 98
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 100
number of neighbors 92
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 99
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 100
number of neighbors 93
number of neighbors 88
number of neighbors 100
number of neighbors 100
number of neighbors 99
number of neighbors 100
number of neighbors 84
number of neighbors 100
number of neighbors 100
number of neighbors 86
number of neighbors 97
number of neighbors 100
num

100-element Array{Any,1}:
 ((334, 5), 0.0711102)
 ((334, 5), 0.0711823)
 ((334, 5), 0.0711184)
 ((334, 5), 0.0711638)
 ((334, 5), 0.0710995)
 ((334, 5), 0.0716698)
 ((334, 5), 0.0724431)
 ((334, 5), 0.0716635)
 ((334, 5), 0.0718935)
 ((334, 5), 0.0710867)
 ((9, 5), 0.0720622)  
 ((9, 5), 0.0713815)  
 ((9, 5), 0.0712261)  
 ⋮                    
 ((123, 5), 0.0715904)
 ((123, 5), 0.0710262)
 ((478, 5), 0.0718798)
 ((478, 5), 0.0709836)
 ((478, 5), 0.0709568)
 ((478, 5), 0.0710015)
 ((478, 5), 0.071075) 
 ((478, 5), 0.0711051)
 ((478, 5), 0.0716479)
 ((478, 5), 0.0710175)
 ((478, 5), 0.0710508)
 ((478, 5), 0.0710356)

In [39]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[n[1]] += v
end
sum_scores
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(9, 0.781752)  
 Pair{Any,Any}(35, 0.781151) 
 Pair{Any,Any}(414, 0.780897)
 Pair{Any,Any}(334, 0.780593)
 Pair{Any,Any}(478, 0.779512)
 Pair{Any,Any}(164, 0.778495)
 Pair{Any,Any}(123, 0.77849) 
 Pair{Any,Any}(428, 0.776469)
 Pair{Any,Any}(291, 0.761676)
 Pair{Any,Any}(250, 0.724981)

In [42]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[n] += v
end
sum_scores
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}((414, 5), 0.718281)
 Pair{Any,Any}((334, 5), 0.714431)
 Pair{Any,Any}((35, 5), 0.71414)  
 Pair{Any,Any}((9, 5), 0.713123)  
 Pair{Any,Any}((123, 5), 0.711902)
 Pair{Any,Any}((428, 5), 0.711783)
 Pair{Any,Any}((478, 5), 0.711754)
 Pair{Any,Any}((164, 5), 0.711648)
 Pair{Any,Any}((291, 5), 0.695053)
 Pair{Any,Any}((250, 5), 0.657184)

In [31]:
accu_scores = DefaultDict([0.]) 
for (n, v) in scores
#     println("node $n")
#     println("value $v")
    push!(accu_scores[node_key(n)], v)
#     push!(accu_scores[node_key(n)], accu_scores[node_key(n)][end] + v)
end
accu_scores

node TimeNode(11, 1)
value 1224.834899819936
node TimeNode(17, 1)
value 1163.8518998292761
node TimeNode(45, 1)
value 1293.5838998093886
node TimeNode(52, 1)
value 906.0639998690834
node TimeNode(25, 1)
value 778.2052998885803
node TimeNode(34, 1)
value 1238.168899817753
node TimeNode(36, 1)
value 1539.8606997719116
node TimeNode(54, 1)
value 1144.227099832334
node TimeNode(55, 1)
value 1169.1183998284694
node TimeNode(43, 1)
value 1099.1326998390223
node TimeNode(80, 1)
value 1363.896299798577
node TimeNode(23, 1)
value 1047.9910998470273
node TimeNode(6, 1)
value 1296.9582998088517
node TimeNode(37, 1)
value 1228.5242998193146
node TimeNode(46, 1)
value 910.8328998682736
node TimeNode(60, 1)
value 1141.6199998327547
node TimeNode(89, 1)
value 1341.6504998016464
node TimeNode(33, 1)
value 922.5920998664808
node TimeNode(73, 1)
value 900.4506998700047
node TimeNode(94, 1)
value 1128.5436998347964
node TimeNode(11, 2)
value 746.1284998952133
node TimeNode(17, 2)
value 910.4348998706787


DataStructures.DefaultDict{Any,Any,Array{Float64,1}} with 0 entries

In [30]:
using Plots

INFO: Precompiling module Plots.


In [26]:
r2 = collect(sum_scores);

In [27]:
sorted_r2 = sort(r2, by = x -> x[2], rev = true)

20-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(2, 3596.32)  
 Pair{Any,Any}(58, 3370.67) 
 Pair{Any,Any}(7, 3226.47)  
 Pair{Any,Any}(11, 3048.78) 
 Pair{Any,Any}(88, 3047.22) 
 Pair{Any,Any}(18, 2958.37) 
 Pair{Any,Any}(57, 2957.42) 
 Pair{Any,Any}(65, 2942.76) 
 Pair{Any,Any}(32, 2939.11) 
 Pair{Any,Any}(97, 2930.67) 
 Pair{Any,Any}(33, 2926.23) 
 Pair{Any,Any}(45, 2879.59) 
 Pair{Any,Any}(87, 2832.12) 
 Pair{Any,Any}(10, 2771.23) 
 Pair{Any,Any}(100, 2756.39)
 Pair{Any,Any}(68, 2746.95) 
 Pair{Any,Any}(44, 2738.5)  
 Pair{Any,Any}(83, 2672.8)  
 Pair{Any,Any}(74, 2669.7)  
 Pair{Any,Any}(26, 2452.59) 

In [28]:
sorted_top

20-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(32), 0.320363) 
 (Node(58), 0.244402) 
 (Node(100), 0.235876)
 (Node(2), 0.199925)  
 (Node(74), 0.198615) 
 (Node(83), 0.162692) 
 (Node(26), 0.152276) 
 (Node(97), 0.138456) 
 (Node(68), 0.137589) 
 (Node(18), 0.126655) 
 (Node(7), 0.120336)  
 (Node(57), 0.108358) 
 (Node(10), 0.103718) 
 (Node(44), 0.099909) 
 (Node(45), 0.08922)  
 (Node(87), 0.0857851)
 (Node(33), 0.0857679)
 (Node(88), 0.0825459)
 (Node(11), 0.0791009)
 (Node(65), 0.0738873)

In [43]:
using EvolvingGraphs
using EvolvingGraphs.Centrality
import EzXML


# load graph data at each year
# form evolving graph
g = EvolvingGraph{Node{String}, Int}()

for year in range(2001, 17)
    sg = load_graphml("jmlr_$(year).graphml")
    add_graph!(g, sg, year)
end
g

Directed EvolvingGraph 3354 nodes, 15654 static edges, 17 timestamps

In [50]:
rating = katz(g, 0.2)


sorted_authors = sort(rating, by = x -> x[2], rev = true)

println("Top 10 rating authors")
sorted_authors[1:10]
nodes(g)

Top 10 rating authors


3354-element Array{EvolvingGraphs.Node{String},1}:
 Node(Manevitz, Larry M.)      
 Node(Yousef, Malik)           
 Node(Gates, Kevin E.)         
 Node(Masters, Annette)        
 Node(Downs, Tom)              
 Node(Vapnik, Vladimir)        
 Node(Siegelmann, Hava T.)     
 Node(Ben-Hur, Asa)            
 Node(Horn, David)             
 Node(Singer, Yoram)           
 Node(Crammer, Koby)           
 Node(Genton, Marc G.)         
 Node(Pekalska, Elzbieta)      
 ⋮                             
 Node(Paul Rochet)             
 Node(Hanwen Huang)            
 Node(Niharika Challapalli)    
 Node(Mathukumalli Vidyasagar) 
 Node(Deepayan Chakrabarti)    
 Node(Sofus A. Macskassy)      
 Node(Jonathan Chang)          
 Node(Stanislav Funiak)        
 Node(Federico Ricci-Tersenghi)
 Node(Jack Raymond)            
 Node(Martin S. Copenhaver)    
 Node(Dimitris Bertsimas)      

In [ ]:
scores = []

for n in 1:10
    for t in 1:10
        intn = (n, 5)
        tr = temporal_katz(g, intn, alpha = 0.2, k = 3)
        push!(scores, (intn, tr))
    end
end
scores